![](https://uploads-ssl.webflow.com/62233c592d2a1e009d42f46c/6414802c0a2bea367cbc795b_logo-jedha-square.svg)

<h1 style="text-align: left; color:#20a08d; font-size: 40px"><span><strong>Q Learning dans l'environnement Cart Pole
</strong></span></h1>

Cet exercice se concentrera sur la résolution du problème d'apprentissage par renforcement pour l'environnement Cart Pole.

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span><strong>Description
</strong></span></h2>

Cet environnement correspond à la version du problème du chariot décrit par Barto, Sutton et Anderson dans "Neuronlike Adaptive Elements That Can Solve Difficult Learning Control Problem". 

Un pendule est attaché par une articulation non actionnée à un chariot, qui se déplace le long d'une piste sans frottement. Le pendule est placé debout sur le chariot et le but est d'équilibrer le pendule en appliquant des forces dans la direction gauche et droite sur le chariot.

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span><strong>Espace d'actions
</strong></span></h2>

L'action est un vecteur de forme (1,) qui peut prendre des valeurs {0, 1} indiquant la direction de la force fixe avec laquelle le chariot est poussé.

The action is a ndarray with shape (1,) which can take values {0, 1} indicating the direction of the fixed force the cart is pushed with.

Action:
* 0 : Pousser le chariot vers la gauche
* 1 : Pousser le chariot vers la droite

Remarque : La vitesse qui est réduite ou augmentée par la force appliquée n'est pas fixe et dépend de l'angle vers lequel pointe le pendule. Le centre de gravité du pendule fait varier la quantité d'énergie nécessaire pour déplacer le chariot en dessous


<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span><strong>Espace d'états
</strong></span></h2>

L'observation est un vecteur de forme (4,) avec les valeurs correspondant aux positions et vitesses suivantes :

Num
Observation
Min
Max

* 0: Cart Position $\in [-4.8, 4.8]$
* 1: Cart Velocity $\in [-\infty, \infty]$
* 2: Pole Angle $\in [~ -0.418 rad (-24°), ~ 0.418 rad (24°)]$
* 3: Pole Angular Velocity $\in [-\infty,\infty]$

Remarque : Bien que les plages ci-dessus indiquent les valeurs possibles pour l'espace d'observation de chaque élément, elles ne reflètent pas les valeurs autorisées de l'espace d'état dans un épisode non terminé. Notamment:

- La position x du chariot (index 0) peut prendre des valeurs comprises entre (-4,8, 4,8), mais l'épisode se termine si le panier quitte la plage (-2,4, 2,4).

- L'angle polaire peut être observé entre (-.418, .418) radians (ou ±24°), mais l'épisode se termine si l'angle polaire n'est pas dans la plage (-.2095, .2095) (ou ±12°)


<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span><strong>Récompenses
</strong></span></h2>

Puisque le but est de maintenir le pendule droit le plus longtemps possible, une récompense de +1 pour chaque pas effectué, y compris le pas de fin d'épisode, est attribuée. Le seuil de récompenses est de 475 pour la v1.

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span><strong>Etat de départ
</strong></span></h2>
Toutes les observations se voient attribuer une valeur uniformément aléatoire en (-0,05, 0,05)

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span><strong>Fin de l'épisode
</strong></span></h2>
L'épisode se termine si l'un des événements suivants se produit :

- L'angle du pendule est supérieur à ±12°
- La position du chariot est supérieure à ±2,4 (le centre du chariot atteint le bord de l'écran)
- La longueur de l'épisode est supérieure à 500 steps(200 pour la v0)

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>1. Nous allons commencer par installer les bibliothèques de gym nécessaires à la simulation de l'environnement</strong></span></h3>


In [1]:
#!pip3 install gym
#!pip3 install gym[classic_control]

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>2. Nous procéderons également aux imports nécessaires
</strong></span></h3>


In [2]:
import numpy as np # used for arrays

import gym # pull the environment

import time # to measure execution time

import math # needed for calculations

import matplotlib.pyplot as plt # for visualizing

import pygame # has some effect on the rendering

import matplotlib.animation as animation # for making gifs


<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>3. Créons une variable env utilisant la méthode décrite dans la documentation
</strong></span></h3>

Documentation Car Pole : [documentation](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>4. jetez un oeil à l'espace d'actions.
</strong></span></h3>

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>5. jetez un oeil à l'espace d'états.
</strong></span></h3>

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>6. Réinitialiser l'environnement pour jeter un œil à un état.
</strong></span></h3>

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>7. Puisque l'environnement est continu dans le temps, nous ferons des gifs pour pouvoir visualiser ce que fait l'agent
</strong></span></h3>

Nous allons suivre quelques étapes pour ce faire :
- Réinitialiser l'environnement
- Définissez une liste vide appelée `arr`
- Définir une variable `done` avec une valeur `False`
- Définir une variable `i` avec une valeur `0`
- Démarrez une boucle `while` qui se poursuivra tant que `done` est `False` dans la boucle :
    - Ajouter la visualisation de l'état actuel de l'environnement à la liste`arr` (render_mode='rgb_array'). Le retour de la méthode render devrait être un tableau numpy.
    - Effectuer une action aléatoire pour produire la nouvelle observation (état)
    - Afficher la nouvelle observation (état)

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>8. Utilisez la commande suivante pour créer un gif à partir de la liste arr des rendus (render)
</strong></span></h3>

Puisque l'espace d'observation est continu et non discret, nous ne pouvons pas appliquer directement le Q-learning. L'astuce consiste ici à convertir ce problème d'apprentissage par renforcement à état continu en un problème d'apprentissage par renforcement discret en divisant la plage des différentes métriques d'observation en catégories. Faisons cela

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>9. Quelle est la fourchette légale des quatre mesures différentes qui définissent un état non terminal ?
</strong></span></h3>
Pour faire la fonction de discrétisation cependant, n'hésitez pas à supposer des plages plus grandes pour éviter toute erreur

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>10. Nous aimerions diviser toutes nos variables d'état en 51 catégories, mettre en place un objet Observation  qui est une liste [51,51,51,51]
</strong></span></h3>

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>11. Nous devons définir une fonction (nous l'appellerons get_discrete_state) qui transforme une observation d'état continue en une observation d'état discrète. 
</strong></span></h3>

L'idée est d'avoir 50 catégories avec la catégorie 0 correspondant à la valeur la plus basse et 50 étant la valeur la plus élevée. L'entrée de la fonction doit être un état et la sortie doit être un tuple d'entiers compris entre 0 et 50

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>12. Créez la table Q pour le problème d'apprentissage par renforcement discrétisé, initialisez-la avec des zéros. Quelle est sa forme ?
</strong></span></h3>

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>13. Initialisons maintenant les valeurs dont nous aurons besoin pour que l'algorithme Q-learning fonctionne
</strong></span></h3>

* `LEARNING_RATE` = 0.1
* `DISCOUNT` = 0.95
* `EPISODES` = 60000
* `total` = 0
* `total_reward` = 0
* `prior_reward` = 0
* `epsilon` = 0.1

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>14. Codons maintenant l'algorithme Q-learning, voici les étapes :
</strong></span></h3>

- Boucle sur le nombre d'épisodes
- réinitialiser l'environnement
- discrétiser l'état initial
- initialiser `done=False`
- initialiser `episode_reward=0`
- Faire une boucle jusqu'à ce que `done` soit `True`
    - conditions de mise en place pour mettre en œuvre une politique $\epsilon-greedy$
    - faire un step en réalisant l'action choisie
    - incrémenter `episode_reward`
    - discrétiser le nouvel état
    - si l'état n'est pas terminal :
        - mettre à jour la Q-table avec la règle de mise à jour de Q-learning
    - remplacer l'état discret actuel par le nouvel état discret

En guise de contrôle d'intégrité, affichez la moyenne episode_reward calculée sur les 1 000 derniers épisodes.

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span><strong>15. Maintenant que l'entrainement est terminé, il semble que les choses se soient très bien passées ! Réutilisez le code pour faire l'animation afin d'afficher le comportement de l'agent entrainé sur le jeu CartPole
</strong></span></h3>

Toutes nos félicitations! Vous avez résolu avec succès un problème d'apprentissage par renforcement avec un espace d'état continu ! La technique de discrétisation est très courante face à un espace d'état continu assez simple comme celui-ci ! Une autre approche consisterait à s'appuyer sur l'approximation de fonctions afin de mapper les états et les actions sur différentes valeurs afin de sélectionner des actions à l'aide d'une fonction (et c'est là que les réseaux de neurones profonds peuvent jouer un rôle important !).

NB : Notez que discrétiser un problème comme celui-ci pour utiliser le Q-learning classique est en soi déjà une approximation de fonction ;)